In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
bike= pd.read_csv("day.csv")

In [ ]:
bike.head()

In [ ]:
# Let's see how many rows and columns we have

bike.shape

In [ ]:
#Let's get some information on the dataset

bike.info()

In [ ]:

bike.describe()

# DATA VISUALISATION

### PERFORMING EDA

In [ ]:
#Dropping the redundant variable holiday as the workingday column covers enough information that is required.

bike.drop('holiday',axis=1,inplace=True)



In [ ]:
# Dropping the dteday,instant,casual and registered columns.

bike.drop(['dteday','instant','casual','registered'],axis=1,inplace=True)

In [ ]:
bike.head()

In [ ]:
## Encoding the Labels & Visualization##

In [ ]:
## 1.season:
codes = {1:'spring',2:'summer',3:'fall',4:'winter'}
bike['season'] = bike['season'].map(codes)

In [ ]:
sns.barplot('season','cnt',data=bike)
plt.show()

#### Observation: Bike Rentals are more during the Fall season

In [ ]:
## 2) weathersit:
codes = {1:'Clear',2:'Mist',3:'Light Snow',4:'Heavy Rain'}
bike['weathersit'] = bike['weathersit'].map(codes)

sns.barplot('weathersit','cnt',data=bike)
plt.show()

#### Observation: Bikes seem to be rented more in Partly cloudy weather

In [ ]:
## 3) Workingday
codes = {1:'working_day',0:'Holiday'}
bike['workingday'] = bike['workingday'].map(codes)

sns.barplot('workingday','cnt',data=bike)
plt.show()

In [ ]:
## 4) Month

codes = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
bike['mnth'] = bike['mnth'].map(codes)


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot('mnth','cnt',data=bike)
plt.show()

In [ ]:
## 5) WeekDay

codes = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
bike['weekday'] = bike['weekday'].map(codes)

In [ ]:
bike.groupby('weekday')['cnt'].max().plot(kind='bar')
plt.show()

#### Observation: Bikes seem to be rented more on Monday and Sunday's.

# CORRELATION BETWEEN VARIABLES

###  Visualizing the Relationship among variables

In [ ]:
sns.pairplot(bike)
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(bike.corr(),annot=True)
plt.show()

In [ ]:
data= bike[['temp','atemp','hum','windspeed']]
sns.heatmap(data.corr(),annot=True)
plt.show()

#### observation: It is observed that atemp and temp are highly correlated and one can be dropped to avoid multicollinearity.

In [ ]:
## Dropping 'atemp':

bike.drop('atemp',axis=1,inplace=True)

In [ ]:
bike.head()

# DATA PREPARATION

### Creating Dummy Variables

### Dealing With Categorical Variables

In [ ]:
seasons = pd.get_dummies(bike['season'],drop_first=True)

working_day = pd.get_dummies(bike['workingday'],drop_first=True)

weather= pd.get_dummies(bike['weathersit'],drop_first=True)

mnth= pd.get_dummies(bike['mnth'],drop_first=True)

weekday= pd.get_dummies(bike['weekday'],drop_first=True)

In [ ]:
bike= pd.concat([bike,seasons,working_day,weather,mnth,weekday],axis=1)

In [ ]:
bike.head()

In [ ]:
# Dropping the categorical variables as they are already dummy-encoded.

bike.drop(['season','workingday','weathersit','weekday','mnth'],axis=1,inplace=True)

In [ ]:
bike.head()

### Splitting the Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
df_train, df_test = train_test_split(bike, train_size = 0.7, test_size = 0.3, random_state = 120)

### Rescaling the Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
numerical_scaler = MinMaxScaler()
num_vars = ['temp','hum','windspeed']

numerical_scaler.fit(df_train[num_vars])
df_train[num_vars] = numerical_scaler.transform(df_train[num_vars])

### Dividing into X and y sets for the model building

In [ ]:
y_train = df_train.pop('cnt')
X_train = df_train

# DATA MODELLING AND EVALUATION

## Data Modelling
### Approach
#### A mixed approach is followed.
#### 15 Best columns are chosen using RFE
#### And then p-value method is followed for further elimination.

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

### Creating Linear Regression Model using Mixed Approach:

In [ ]:
# Running RFE with the output number of the variable equal to 15


lr_estimator = LinearRegression()
rfe = RFE(lr_estimator,n_features_to_select=15, step=1)
selector = rfe.fit(X_train,y_train)

In [ ]:
# RFE Feature Ranking
rfe_ranking = pd.DataFrame({'rank' : selector.ranking_, 'support': selector.support_, 'features' : X_train.columns}).sort_values(by='rank',ascending=True)
rfe_ranking

In [ ]:
# Selected Features
selected_features = rfe_ranking.loc[rfe_ranking['rank'] == 1,'features'].values
selected_features

### Manual Elimination

In [ ]:
# Following a stepwise elimination
import statsmodels.api as sm
def ols_fit(y,X) : 
    X_train_sm = sm.add_constant(X)
    model = sm.OLS(y,X_train_sm).fit()
    print(model.summary())
    return model
def vif(X) : 
    df = sm.add_constant(X)
    vif = [variance_inflation_factor(df.values,i) for i in range(df.shape[1])]
    vif_frame = pd.DataFrame({'vif' : vif[0:]},index = df.columns).reset_index()
    print(vif_frame.sort_values(by='vif',ascending=False))

#### Model 1: Using features selected by RFE 

In [ ]:
features_1 = selected_features
ols_fit(y_train,X_train[features_1])
vif(X_train[selected_features])

#### Model 2:  Dropping 'working_day' as VIF > 5:

In [ ]:

del_feature = 'working_day'
selected_features = selected_features[selected_features!=del_feature]
final_model = ols_fit(y_train,X_train[selected_features])
vif(X_train[selected_features])


#### Model 3: Dropping 'Sun' because of high p-value:

In [ ]:

del_feature = 'Sun'
selected_features = selected_features[selected_features!=del_feature]
ols_fit(y_train,X_train[selected_features])
vif(X_train[selected_features])

#### Model 4: Dropping 'Dec' because this information might also be contained in winter :


In [ ]:

del_feature = 'Dec'
selected_features = selected_features[selected_features!=del_feature]
ols_fit(y_train,X_train[selected_features])
vif(X_train[selected_features])

#### Model 5: Dropping 'Nov' because this information might also be contained in winter :

In [ ]:
del_feature = 'Nov'
selected_features = selected_features[selected_features!=del_feature]
ols_fit(y_train,X_train[selected_features])
vif(X_train[selected_features])

#### Model 6: Dropping 'Mon':

In [ ]:
del_feature = 'Mon'
selected_features = selected_features[selected_features!=del_feature]
ols_fit(y_train,X_train[selected_features])
vif(X_train[selected_features])

#### Verifying Multicollinearity:

In [ ]:

vif(X_train[selected_features])

#### Observation: VIF < 5 for selected features. No significant multicollinearity observed. Similar indicating comparison of R-squared and adjusted R-squared.

### Final Model

In [ ]:
final_model = ols_fit(y_train,X_train[selected_features])

#### Observation: 10 features have been selected.
#### All the features are statistically significant [low p-value]
#### The model over is a good fit with Prob (F-statistic): 6.67e-181
#### The model explains 82.4% variability in the training data. Adjusted R-square being 82.0%

### Residual Analysis

In [ ]:
# Residual Analysis of Trained Data
X_train_sm = sm.add_constant(X_train[selected_features])

y_train_pred = final_model.predict(X_train_sm)


fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 30)
fig.suptitle('Frequency Distribution of Residuals', fontsize = 20)  
plt.xlabel('Errors', fontsize = 18) 
plt.show()

# Prediction 


In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test
X_test[num_vars] = numerical_scaler.transform(X_test[num_vars])
X_test = X_test[selected_features]

In [ ]:
X_test_sm = sm.add_constant(X_test[selected_features])

y_test_pred = final_model.predict(X_test_sm)

# Model Evaluation

In [ ]:

# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_test_pred)
fig.suptitle('Actual vs Predictions', fontsize=20)          
plt.xlabel('Actual', fontsize=18)                          
plt.ylabel('Predictions', fontsize=16) 
plt.show()

#### Calculating r2_score

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
rsquared_test = r2_score(y_test, y_test_pred)
rsquared_train = r2_score(y_train, y_train_pred)
print('R-squared for train data:',round(rsquared_train,2))
print('R-squared for test data:',round(rsquared_test,2))

# Conclusion

#### 1) Analysis is carried out using a Mixed Feature Selection Approach. 15 features are selected algorithmically using Recursive Feature Elimination. Further selection is done manually by looking at multicollinearity and statistical significance of features and overall fit of the model. The 10 most significant features to understand demand have been reported.

#### 2) Varibles such as 'temp', 'windspeed' & ' hum' are the one which are affecting the Bike Rentals the most.

#### 3) The data set is randomly divided into training and test data. Final Model built on training data set explains 82% of the variability and achieves 84% on test data.

### Note:

#### The model has been verified for Multicollinearity effects.
#### Residual Analysis has been carried out and the model satisfies the assumptions of Linear Regression (Residuals follow a normal distribution, Errors exhibit homoscedasticity)
#### Features in the order of influence has been reported by standardizing all predictor values.